The following notebook adapts some code from the TensorFlow tutorial: https://www.tensorflow.org/tutorials/keras/classification

In [25]:
import keras
from keras import Sequential
from keras.layers  import Dense, Activation
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Helper libraries
import numpy as np  # Keras needs an older version on numpy
import matplotlib.pyplot as plt
from collections import OrderedDict

In [39]:
from RProp import RProp
from Wame import Wame
from WameMosca import WameMosca

## Import Landsat

In [3]:
def extract_data_lables(file):
    data, lables = [], []
    with open(file, "r") as f:
        for row in f:
            row = row.split()
            row = [int(i) for i in row]
            lables.append(row.pop(-1))
            data.append(row)
        return np.array(data), np.array(lables)

In [4]:
train_data, train_lables = extract_data_lables("data\landsat\sat.trn")

In [5]:
test_data, test_lables  = extract_data_lables("data\landsat\sat.tst")

In [6]:
print(len(train_data))
print(len(test_data))

4435
2000


### Clean Lables

In [7]:
train_lables -= 1
test_lables -= 1

In [8]:
class_names = {
    0:'red soil'
    ,1:'cotton crop'
    ,2:'grey soil'
    ,3:'damp grey soil'
    ,4:'soil with vegetation stubble'
    ,5:'mixture class (all types present)'
    ,6:'very damp grey soil'
}

## Preprocess Images

In [9]:
# scale pixels to 0-1
train_data = train_data / 255.0
test_data = test_data / 255.0

TODO: Find a way to optimise these layers (will be needed for report)

## One-hot encode lables
Class lables need to be in an nxc array format

In [10]:
from sklearn.preprocessing import OneHotEncoder

In [11]:
ohe = OneHotEncoder(sparse=False)
train_y = ohe.fit_transform(train_lables.reshape(-1, 1))
test_y = ohe.transform(test_lables.reshape(-1, 1))

In [12]:
train_data.shape

(4435, 36)

In [13]:
train_y.shape

(4435, 6)

## Functions for creating and testing models

In [14]:
def create_seq_model(n_hidden=1, n_nodes=36):
    model = Sequential()
    # First hidden layer
    model.add(Dense(n_nodes, input_dim=36))  # input_dim easier to use than input_shape
    model.add(Activation('relu'))
    # Additional hidden layers
    for i in range(n_hidden-1):
        model.add(Dense(n_nodes))
        model.add(Activation('relu'))
    # output layer
    model.add(Dense(6))  
    model.add(Activation('sigmoid'))
    return model

In [15]:
def train_test_model(model, optimiser='adam'):
    """Trains a model using 25% validation, then tests the model with the best validation performance"""
    
    model.compile(optimizer=optimiser
              ,loss='categorical_crossentropy'
              ,metrics=['accuracy']
             )
    
    np.random.seed(123)
    
    history =  model.fit(
        train_data
        , train_y
        , epochs=100
        , validation_split=0.25
        , verbose=0
        , batch_size=100
        ,callbacks = [
            EarlyStopping(verbose=False, patience=20, monitor='val_acc'),
            ModelCheckpoint('TestModel-progress', monitor='val_acc', verbose=False, save_best_only=True)],
    )
    train_loss, train_acc = model.evaluate(train_data,  train_y, verbose = 0)
    test_loss, test_acc = model.evaluate(test_data,  test_y, verbose = 0)
    
    epochs = history.history['val_acc']
    best_epoch = epochs.index(max(epochs))
    
    return (train_acc, test_acc, best_epoch, history)   

In [31]:
def train_test_grid(n_hidden_lst, n_nodes_lst, optimiser, n_experiments=20):
    results = []
    
    if isinstance(optimiser, str):
        opt_alias = optimiser
    else:
        opt_alias = type(optimiser).__name__

    for i, n_hidden in enumerate(n_hidden_lst):
        for j, n_nodes in enumerate(n_nodes_lst):
            record_dict = OrderedDict({
                'hidden_layer': n_hidden
                ,'nodes': n_nodes
                ,'optimiser': opt_alias
                ,'n_experiments': n_experiments
                ,'train_accuracies': []
                ,'test_accuracies': []
                ,'best_epochs': []
            })
            
            for k in range(n_experiments):
                model = create_seq_model(n_hidden, n_nodes)
                train_acc, test_acc, best_epoch, _ = train_test_model(model, optimiser)
                record_dict['train_accuracies'].append(train_acc)
                record_dict['test_accuracies'].append(test_acc)
                record_dict['best_epochs'].append(best_epoch)
            
            record_dict['avg_train_acc'] = sum(record_dict['train_accuracies'])/len(record_dict['train_accuracies'])
            record_dict['avg_test_acc'] = sum(record_dict['test_accuracies'])/len(record_dict['test_accuracies'])
            record_dict['avg_epoch'] = sum(record_dict['best_epochs'])/len(record_dict['best_epochs'])
            
            results.append(record_dict)
            
    
    return results

##### Test various 'wame' configurations

In [32]:
n_hidden_lst = [1, 2]
n_nodes_lst = [20, 30]

In [33]:
wame_results = train_test_grid(n_hidden_lst, n_nodes_lst, Wame(), n_experiments=5)

In [36]:
wame_results[1]

OrderedDict([('hidden_layer', 1),
             ('nodes', 20),
             ('optimiser', 'Wame'),
             ('n_experiments', 5),
             ('train_accuracies',
              [0.23404735062006765,
               0.2342728297632469,
               0.23404735062006765,
               0.23404735062006765,
               0.23404735062006765]),
             ('test_accuracies', [0.235, 0.235, 0.235, 0.235, 0.235]),
             ('best_epochs', [2, 20, 2, 21, 0]),
             ('avg_train_acc', 0.23409244644870347),
             ('avg_test_acc', 0.23499999999999996),
             ('avg_epoch', 9.0)])

In [40]:
wame_mosca_results = train_test_grid(n_hidden_lst, n_nodes_lst, WameMosca(), n_experiments=5)

In [41]:
wame_mosca_results

[OrderedDict([('hidden_layer', 1),
              ('nodes', 10),
              ('optimiser', 'WameMosca'),
              ('n_experiments', 5),
              ('train_accuracies',
               [0.8101465614901051,
                0.813528748604195,
                0.8121758737787185,
                0.8124013529218977,
                0.8124013529218977]),
              ('test_accuracies', [0.789, 0.7925, 0.794, 0.794, 0.794]),
              ('best_epochs', [34, 16, 16, 16, 16]),
              ('avg_train_acc', 0.8121307779433629),
              ('avg_test_acc', 0.7927000000000001),
              ('avg_epoch', 19.6)]),
 OrderedDict([('hidden_layer', 1),
              ('nodes', 20),
              ('optimiser', 'WameMosca'),
              ('n_experiments', 5),
              ('train_accuracies',
               [0.8189402480136179,
                0.8160090192463648,
                0.8187147688704386,
                0.8160090192463648,
                0.8189402480136179]),
              (

### Scratch:

In [ ]:
test_losses_wame

In [ ]:
epochs = history.history['val_acc']
epochs.index(max(epochs))

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'Val'], loc='upper left')
plt.show()